In [2]:
import os
import sys
import pandas as pd
import argparse
from datetime import datetime, timedelta
import pathlib

# 新增：將上一層目錄加入 sys.path
parent_path = pathlib.Path().absolute().parent
sys.path.insert(0, str(parent_path))
# Setup Django environment
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'website_configs.settings')
import django
django.setup()
# 重要：設定環境變數以允許在 Jupyter 的異步環境中執行同步操作
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Now we can import Django models
from app_user_keyword_db.models import NewsData

# Django ORM CRUD Operations Tutorial

CRUD stands for Create, Read, Update, and Delete - the four basic operations for database management systems. In this tutorial, we'll explore how to perform these operations using Django ORM with the NewsData model.

## NewsData Model Structure

Let's first examine the structure of our NewsData model to understand its fields.

In [3]:
# Inspect the NewsData model fields
print('NewsData Model Fields:')
for field in NewsData._meta.get_fields():
    print(f'- {field.name}: {field.get_internal_type()}')

# Display total count of news entries
print(f'\nTotal news entries in database: {NewsData.objects.count()}')

NewsData Model Fields:
- item_id: CharField
- date: DateField
- category: CharField
- title: TextField
- content: TextField
- sentiment: FloatField
- top_key_freq: TextField
- tokens: TextField
- tokens_v2: TextField
- entities: TextField
- token_pos: TextField
- link: CharField
- photo_link: CharField

Total news entries in database: 957


## 1. Create (C) - Adding New Data

The 'Create' operation allows us to insert new records into the database. Here are different ways to create new NewsData objects.

In [4]:
# Method 1: Create using the constructor and save() method
new_news1 = NewsData(
    title="馬斯克蒞臨高科大",
    link="https://example.com/news1",
    category="測試類別",
    content="This is a sample content about technology news for demonstration.",
    date=datetime.now().date(),
    photo_link="https://example.com/image1.jpg"
)
# Uncomment to save: new_news1.save()

# Method 2: Using objects.create() method
# This creates and saves in one step
new_news2 = NewsData.objects.create(
    title="馬斯克蒞臨高科大",
    link="https://example.com/news2",
    category="測試類別",
    content="This is sample content about international news for demonstration.",
    date=datetime.now().date(),
    photo_link=None  # Shows that photo_link can be optional
)

# Method 3: Bulk creation (for adding multiple records at once)
news_list = [
    NewsData(
        title=f"Bulk Example {i}", 
        link=f"https://example.com/bulk{i}",
        category="測試類別",
        content=f"Bulk created content {i}",
        date=datetime.now().date()
    ) for i in range(3, 6)
]
# Uncomment to save: NewsData.objects.bulk_create(news_list)

print(f"Created news item: {new_news2.title} (ID: {new_news2.item_id})")
print("Note: The other examples are commented out to prevent adding test data to your database.")

Created news item: 馬斯克蒞臨高科大 (ID: )
Note: The other examples are commented out to prevent adding test data to your database.


In [5]:

# Clean up - delete the example we created
new_news2.delete()

(1, {'app_user_keyword_db.NewsData': 1})

## 2. Read (R) - Querying Data

The 'Read' operation retrieves data from the database. Django ORM provides powerful ways to query data.

In [6]:
# Basic querying examples

# 1. Get all records
all_news = NewsData.objects.all()
print(f"Total news items: {all_news.count()}")


Total news items: 957


In [7]:

# 2. Get a specific record by ID
try:
    first_id = NewsData.objects.first().item_id
    specific_news = NewsData.objects.get(item_id=first_id)
    print(f"\nNews by ID {first_id}: {specific_news.title}")
except:
    print("Couldn't retrieve news by ID")



News by ID CHY_01: 賞螢不必跑山區！嘉義親子路線賞夢幻螢火蟲，特色原民風味餐一次玩。


In [8]:

# 3. Filter records
# Get news from a specific category
tech_news = NewsData.objects.filter(category="科技")
print(f"\nTechnology news count: {tech_news.count()}")
if tech_news.exists():
    print(f"Example tech news: {tech_news.first().title}")



Technology news count: 0


In [9]:

# 4. Complex filtering
# Get news from the last 7 days
one_week_ago = datetime.now().date() - timedelta(days=7)
recent_news = NewsData.objects.filter(date__gte=one_week_ago)
print(f"\nNews from the last 7 days: {recent_news.count()}")



News from the last 7 days: 0


In [10]:

# 5. Ordering
latest_news = NewsData.objects.order_by('-date')[:5]  # Get 5 most recent news
print("\n5 most recent news:")
for news in latest_news:
    print(f"- {news.item_id} {news.date}:{news.category} {news.title}")



5 most recent news:
- TPE_01 2025-04-11:台北市 2025大年初二當天，溫暖如初夏的台北【心中山線形公園】【赤峰街】，悠閒街景速寫
- MAL_01 2025-04-11:苗栗縣 童萌農場 ｜ 苗栗親子景點．近距離和超多小動物互動．還可以抱小小羊！
- TXG_01 2025-04-11:台中市 【台中住宿】永豐棧酒店-經典雙床房入住心得分享，清潔度有待加強，兒童遊戲室玩需額外付費，停車位有限量，早餐味道中規中矩｜Ruby說美食享受旅行(@tour_ruby530)
- TXG_02 2025-04-11:台中市 親子Fun電首選！麗寶追風奇幻島魔鬼滑梯、平衡挑戰樂翻孩童 憑票根嚐美味。
- CWH_01 2025-04-11:彰化縣 彰化縣溪洲鄉彰化日式景點推薦｜不用出國也可以感受到滿滿禪風


In [11]:

# 6. Specific fields selection
titles_only = NewsData.objects.values_list('title')[:3]
print("\nSome news titles:")
for title in titles_only:
    print(f"- {title}")



Some news titles:
- ('基隆市安樂區基隆景點/大武崙山、大武崙砲台、情人湖公園，半日遊很愜意！',)
- ('基隆美食景點9大推薦！基隆人私藏一日遊攻略，爆夯石頭火鍋 天空步道必去。',)
- ('基隆嶼4/1開放登島！最美零死角海景限量名額開放 超療癒海景步道、燈塔美拍必去。',)


In [12]:
titles_only

<QuerySet [('基隆市安樂區基隆景點/大武崙山、大武崙砲台、情人湖公園，半日遊很愜意！',), ('基隆美食景點9大推薦！基隆人私藏一日遊攻略，爆夯石頭火鍋 天空步道必去。',), ('基隆嶼4/1開放登島！最美零死角海景限量名額開放 超療癒海景步道、燈塔美拍必去。',)]>

In [13]:

# 6. Specific fields selection
titles_only = NewsData.objects.values_list('title', flat=True)[:3]
print("\nSome news titles:")
for title in titles_only:
    print(f"- {title}")



Some news titles:
- 基隆市安樂區基隆景點/大武崙山、大武崙砲台、情人湖公園，半日遊很愜意！
- 基隆美食景點9大推薦！基隆人私藏一日遊攻略，爆夯石頭火鍋 天空步道必去。
- 基隆嶼4/1開放登島！最美零死角海景限量名額開放 超療癒海景步道、燈塔美拍必去。


In [14]:

# 7. Text search (simple contains)
keyword = "台灣"
matching_news = NewsData.objects.filter(content__contains=keyword).count()
print(f"\nNews mentioning '{keyword}': {matching_news}")


News mentioning '台灣': 367


In [15]:
from django.db.models import Q, Max, F
from collections import Counter

## 3. Update (U) - Modifying Data

The 'Update' operation modifies existing records in the database. There are several ways to update data in Django.

In [16]:
# Let's demonstrate updates with a temporary test record
test_news = NewsData.objects.create(
    title="Test News for Update Demo",
    link="https://example.com/test-update",
    category="測試",
    content="This is a temporary record to show update operations.",
    date=datetime.now().date()
)


In [17]:

# Method 1: Instance update
print(f"Original title: {test_news.title}")
test_news.title = "Updated Test News Title"
test_news.category = "更新測試"
test_news.save()
print(f"Updated title: {test_news.title}, Category: {test_news.category}")


Original title: Test News for Update Demo
Updated title: Updated Test News Title, Category: 更新測試


In [18]:

# Method 2: QuerySet update (more efficient for multiple records)
# This updates all matching records without loading them into memory
update_count = NewsData.objects.filter(item_id=test_news.item_id).update(
    content="This content was updated using QuerySet.update() method.",
    photo_link="https://example.com/updated-image.jpg"
)
print(f"\nRecords updated via QuerySet: {update_count}")



Records updated via QuerySet: 1


In [19]:
test_news.delete()

(1, {'app_user_keyword_db.NewsData': 1})

## 4. Delete (D) - Removing Data

The 'Delete' operation removes records from the database. Django provides various methods to delete data.

In [20]:
# Let's demonstrate updates with a temporary test record
test_news = NewsData.objects.create(
    item_id="item-123",  # This will be auto-generated by the database
    title="Test News for Update Demo",
    link="https://example.com/test-update",
    category="刪除測試",
    content="This is a temporary record to show update operations.",
    date=datetime.now().date()
)


In [21]:
NewsData.objects.get(item_id="item-123").delete()

(1, {'app_user_keyword_db.NewsData': 1})

In [22]:
NewsData.objects.filter( category="刪除測試")

<QuerySet []>

In [23]:
NewsData.objects.filter( category="刪除測試").delete()

(0, {})

# filter_database_fullText

In [24]:
# Searching keywords from "content" column
# This function now uses database queries instead of pandas
def filter_database_fullText(user_keywords, cond, cate, weeks):
    # Get the latest date in the database
    latest_date = NewsData.objects.aggregate(max_date=Max('date'))['max_date']
    
    # Calculate start date
    start_date = latest_date - timedelta(weeks=weeks)
    
    # Base query - filter by date range
    queryset = NewsData.objects.filter(date__gte=start_date, date__lte=latest_date)
    
    # Filter by category if not "全部"
    if cate != "全部":
        queryset = queryset.filter(category=cate)
    
    # Filter by keywords based on condition (AND or OR)
    if cond == 'and':
        # For AND condition, we need all keywords to be present
        for kw in user_keywords:
            queryset = queryset.filter(content__contains=kw)
    elif cond == 'or':
        # For OR condition, any keyword can be present
        q_objects = Q()
        for kw in user_keywords:
            q_objects |= Q(content__contains=kw)
        # q_objects會長這樣： Q(content__contains=kw1) | Q(content__contains=kw2) | Q(content__contains=kw3)
        # 這樣就可以用在filter裡面了
        queryset = queryset.filter(q_objects)
    
    return queryset

In [25]:

user_keywords = ['烏克蘭', '俄羅斯']  # Example keywords
cond = 'and'  # Example condition (and/or), 
cate = '全部'  # Example category (or "全部" for all categories)
weeks = 4  # Example weeks
queryset = filter_database_fullText(user_keywords, cond, cate, weeks)

In [26]:
for news in queryset:
    print(news.title)

In [27]:
# Limit to k results and get specific fields
news_items = queryset.values('category', 'title', 'link', 'photo_link')[:3]

In [28]:
news_items

<QuerySet []>

In [29]:
list(queryset.values_list('date', flat=True)) # flat=True to get a flat list 否則會是 tuple

[]

In [30]:
latest_date = NewsData.objects.aggregate(max_date=Max('date'))['max_date']

In [31]:
latest_date

datetime.date(2025, 4, 11)

In [32]:
from django.db.models import Avg, Count, Min, Max, Sum
from django.db.models.functions import Extract

# Aggregate是用來計算總和、平均值、最小值、最大值等統計數據的函數
# 這些函數可以用來對查詢集進行聚合計算，並返回一個字典，其中包含計算結果的鍵值對
# Basic aggregation examples
aggregations = NewsData.objects.aggregate(
    count=Count('item_id'),
    oldest=Min('date'),
    newest=Max('date'),
)
print("Basic aggregations:")
for key, value in aggregations.items():
    print(f"- {key}: {value}")


Basic aggregations:
- count: 957
- oldest: 2023-08-20
- newest: 2025-04-11


In [33]:

# Category-based aggregation
category_counts = NewsData.objects.values('category').annotate(
    count=Count('item_id')
).order_by('-count')[:5]  # Top 5 categories
print("\nTop 5 categories by count:")
for item in category_counts:
    print(f"- {item['category']}: {item['count']} articles")


Top 5 categories by count:
- 台南市: 50 articles
- 高雄市: 49 articles
- 澎湖縣: 49 articles
- 新竹縣: 49 articles
- 新北市: 49 articles


In [34]:
# annotate是用來在查詢集中添加計算字段的函數
# 這些計算字段可以用來進行進一步的過濾、排序或分組操作
# values是用來選擇查詢集中的特定字段的函數 字段就是資料庫中的欄位
# Date-based aggregation (articles per day)
date_counts = NewsData.objects.values('date').annotate(
    count=Count('item_id')
).order_by('-date')[:7]  # Last 7 days with data

print("\nArticles per day (most recent 7 days with data):")
for item in date_counts:
    print(f"- {item['date']}: {item['count']} articles")



Articles per day (most recent 7 days with data):
- 2025-04-11: 10 articles
- 2025-04-10: 13 articles
- 2025-04-09: 6 articles
- 2025-04-08: 20 articles
- 2025-04-07: 13 articles
- 2025-04-06: 4 articles
- 2025-04-05: 4 articles
